In [ ]:
import spacy

!python -m spacy download pt_core_news_lg
pln = spacy.load('pt_core_news_lg')
pln

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
caminho_arquivo = '/content/drive/MyDrive/TCC - UFMS/CSBC-WICS/Corpus/corpus.csv'
dataframe = pd.read_csv(caminho_arquivo)
# Filtrar o DataFrame para excluir as letras do gênero gospel
dataframe_sem_gospel = dataframe[dataframe['genre'] != 'gospel']
# Exibir as primeiras linhas do DataFrame filtrado
dataframe_sem_gospel.head()

,artist_name,genre,music_title,music_lyric
0,Black Style,axe,Amassa a Latinha Com a Bunda,Eu tô trazendo a novidade em salvador\n(eu tô ...
1,Black Style,axe,Vaco Vuco,Você marcou comigo pra sair\nMas já pensou se ...
2,Black Style,axe,"Tome, Todinha",Que mulher gulosa\nEla entra na roda e não que...
3,Black Style,axe,Toma Que É de Graça,"Toma, toma meu carinho\ntoma meu amor e a minh..."
4,Black Style,axe,Toda de Mel,"Foi esculpida,escolhida,desenhada no papel\nNã..."


In [4]:
from spacy.matcher import Matcher

# Create a Matcher and provide model vocabulary; assign result under the variable 'matcher'
matcher = Matcher(vocab=pln.vocab)

# Define a list with nested dictionaries that contains the pattern to be matched
#Você + Auxiliar + Adjetivo (Masculino)
pronoun_aux_adj = [{'POS': 'PRON', "MORPH": {"IN": ['Case=Nom|Number=Sing|Person=3|PronType=Prs']}}, {'POS': 'AUX'}, {'POS': 'ADJ', "MORPH": {"IN":["Gender=Masc|Number=Plur", "Gender=Masc|Number=Sing"]}}] #Você é cheiroso.
#Sujeito + Auxiliar + Adjetivo
sujeito_aux_adj = [{"LOWER": {"IN": ["homem", "homens", "ele", "eles", "menino", "meninos", "garoto", "garotos", "senhor", "senhores", "moços", "moços", "rapaz", "rapazes", "joão", "cavalheiro", "cavalheiros", "rei", "reis", "esposo", "esposos", "marido", "maridos", "namorado", "namorados"]}}, {'POS': 'AUX'}, {'POS': 'ADJ'}]

#Você + Auxiliar + muito/pouco + Adjetivo
pronoun_aux_adv_adj = [{'POS': 'PRON', "MORPH": {"IN": ['Case=Nom|Number=Sing|Person=3|PronType=Prs']}}, {'POS': 'AUX'}, {'POS': 'ADV', 'OP': '+'}, {'POS': 'ADJ', "MORPH": {"IN":["Gender=Masc|Number=Plur", "Gender=Masc|Number=Sing"]}}] # Ela é muito [pouco] inteligente.
#Sujeito + Auxiliar + muito/pouco + Adjetivo
sujeito_aux_adv_adj = [{"LOWER": {"IN": ["homem", "homens", "ele", "eles", "menino", "meninos", "garoto", "garotos", "senhor", "senhores", "moços", "moços", "rapaz", "rapazes", "joão", "cavalheiro", "cavalheiros", "rei", "reis", "esposo", "esposos", "marido", "maridos", "namorado", "namorados"]}}, {'POS': 'AUX'}, {'POS': 'ADV', 'OP': '+'}, {'POS': 'ADJ'}] # Ela é muito [pouco] inteligente.
#Voce + Auxiliar + Auxiliar + Adjetivo (Masculino)

pronoun_aux_aux_adj = [{'POS': 'PRON', "MORPH": {"IN": ['Case=Nom|Number=Sing|Person=3|PronType=Prs']}}, {'POS': 'AUX'}, {'POS': 'AUX'}, {'POS': 'ADJ', "MORPH": {"IN":["Gender=Masc|Number=Plur", "Gender=Masc|Number=Sing"]}}]
#Sujeito + Auxiliar + Auxiliar + Adjetivo
sujeito_aux_aux_adj = [{"LOWER": {"IN": ["homem", "homens", "ele", "eles", "menino", "meninos", "garoto", "garotos", "senhor", "senhores", "moços", "moços", "rapaz", "rapazes", "joão", "cavalheiro", "cavalheiros", "rei", "reis", "esposo", "esposos", "marido", "maridos", "namorado", "namorados"]}}, {'POS': 'AUX'}, {'POS': 'AUX'}, {'POS': 'ADJ'}]
#Voce + Auxiliar + Auxiliar + Muito/pouco + Adjetivo (Masculino)
pronoun_aux_aux_adv_adj = [{'POS': 'PRON',"MORPH": {"IN": ['Case=Nom|Number=Sing|Person=3|PronType=Prs']}}, {'POS': 'AUX'}, {'POS': 'AUX'}, {'POS': 'ADV', 'OP': '+'}, {'POS': 'ADJ', "MORPH": {"IN":["Gender=Masc|Number=Plur", "Gender=Masc|Number=Sing"]}}]
#Sujeito + Auxiliar + Auxiliar + muito/pouco + Adjetivo
sujeito_aux_aux_adv_adj = [{"LOWER": {"IN": ["homem", "homens", "ele", "eles", "menino", "meninos", "garoto", "garotos", "senhor", "senhores", "moços", "moços", "rapaz", "rapazes", "joão", "cavalheiro", "cavalheiros", "rei", "reis", "esposo", "esposos", "marido", "maridos", "namorado", "namorados"]}}, {'POS': 'AUX'}, {'POS': 'ADV', 'OP': '+'}, {'POS': 'ADJ'}]
#Sujeito + Auxiliar + Verbo + Adjetivo
sujeito_aux_verb_adj = [{"LOWER": {"IN": ["homem", "homens", "ele", "eles", "menino", "meninos", "garoto", "garotos", "senhor", "senhores", "moços", "moços", "rapaz", "rapazes", "joão", "cavalheiro", "cavalheiros", "rei", "reis", "esposo", "esposos", "marido", "maridos", "namorado", "namorados"]}}, {'POS': 'AUX'}, {"POS":"VERB"}, {'POS': 'ADJ'}]
#Sujeito + Auxiliar + Verbo + muito/pouco + Adjetivo
sujeito_aux_verb_adv_adj = [{"LOWER": {"IN": ["homem", "homens", "ele", "eles", "menino", "meninos", "garoto", "garotos", "senhor", "senhores", "moços", "moços", "rapaz", "rapazes", "joão", "cavalheiro", "cavalheiros", "rei", "reis", "esposo", "esposos", "marido", "maridos", "namorado", "namorados"]}}, {'POS': 'AUX'}, {"POS":"VERB"}, {'POS': "ADV", 'OP': '+'}, {'POS': 'ADJ'}]
#Voce + Auxiliar + Verbo + Adjetivo (Masculino)
pronoun_aux_verb_adj = [{'POS': 'PRON', "MORPH": {"IN": ['Case=Nom|Number=Sing|Person=3|PronType=Prs']}}, {'POS': 'AUX'}, {"POS":"VERB"}, {'POS': 'ADJ', "MORPH": {"IN":["Gender=Masc|Number=Plur", "Gender=Masc|Number=Sing"]}}] # Ele está ficando velho.
#Voce + Auxiliar + Verbo + muito/pouco + Adjetivo (Masculino)
pronoun_aux_verb_adv_adj = [{'POS': 'PRON', "MORPH": {"IN": ['Case=Nom|Number=Sing|Person=3|PronType=Prs']}}, {'POS': 'AUX'}, {"POS":"VERB"}, {'POS': "ADV", 'OP': '+'}, {'POS': 'ADJ', "MORPH": {"IN":["Gender=Masc|Number=Plur", "Gender=Masc|Number=Sing"]}}]
#Sujeito + Verbo + Auxiliar + Adjetivo
sujeito_verb_aux_adj = [{"LOWER": {"IN": ["homem", "homens", "ele", "eles", "menino", "meninos", "garoto", "garotos", "senhor", "senhores", "moços", "moços", "rapaz", "rapazes", "joão", "cavalheiro", "cavalheiros", "rei", "reis", "esposo", "esposos", "marido", "maridos", "namorado", "namorados"]}}, {'POS': 'VERB'}, {'POS': 'AUX'}, {'POS': 'ADJ'}]
#Sujeito + Verbo + Auxiliar + muito/pouco + Adjetivo
sujeito_verb_aux_adv_adj = [{"LOWER": {"IN": ["homem", "homens", "ele", "eles", "menino", "meninos", "garoto", "garotos", "senhor", "senhores", "moços", "moços", "rapaz", "rapazes", "joão", "cavalheiro", "cavalheiros", "rei", "reis", "esposo", "esposos", "marido", "maridos", "namorado", "namorados"]}}, {'POS': 'VERB'}, {'POS': 'AUX'}, {'POS': "ADV", 'OP': '+'}, {'POS': 'ADJ'}]
#Você + Verbo + Auxiliar + Adjetivo (Masculino)
pronoun_verb_aux_adj = [{'POS': 'PRON',"MORPH": {"IN": ['Case=Nom|Number=Sing|Person=3|PronType=Prs']}}, {'POS': 'VERB'}, {'POS': 'AUX'}, {'POS': 'ADJ', "MORPH": {"IN":["Gender=Masc|Number=Plur", "Gender=Masc|Number=Sing"]}}]
#Você + Verbo + Auxiliar + muito/pouco + Adjetivo (Masculino)
pronoun_verb_aux_adv_adj = [{'POS': 'PRON',"MORPH": {"IN": ['Case=Nom|Number=Sing|Person=3|PronType=Prs']}}, {'POS': 'VERB'}, {'POS': 'AUX'}, {'POS': "ADV", 'OP': '+'}, {'POS': 'ADJ', "MORPH": {"IN":["Gender=Masc|Number=Plur", "Gender=Masc|Number=Sing"]}}]
#Sujeito + Auxiliar(Ser/Estar) + verbo principal no particípio passado
sujeito_aux_verb = [{"LOWER": {"IN": ["homem", "homens", "ele", "eles", "menino", "meninos", "garoto", "garotos", "senhor", "senhores", "moços", "moços", "rapaz", "rapazes", "joão", "cavalheiro", "cavalheiros", "rei", "reis", "esposo", "esposos", "marido", "maridos", "namorado", "namorados"]}}, {'POS': 'AUX'}, {'POS': 'VERB', "MORPH": "Gender=Masc|Number=Sing|VerbForm=Part|Voice=Pass"}]
#Sujeito + Auxiliar(Ser/Estar) + muito/pouco + verbo principal no particípio passado
sujeito_aux_adv_verb = [{"LOWER": {"IN": ["homem", "homens", "ele", "eles", "menino", "meninos", "garoto", "garotos", "senhor", "senhores", "moços", "moços", "rapaz", "rapazes", "joão", "cavalheiro", "cavalheiros", "rei", "reis", "esposo", "esposos", "marido", "maridos", "namorado", "namorados"]}}, {'POS': 'AUX'}, {'POS': "ADV", 'OP': '+'}, {'POS': 'VERB', "MORPH": "Gender=Masc|Number=Sing|VerbForm=Part|Voice=Pass"}]
#Sujeito + Verbo + Adjetivo / LEMMA tornar, ficar, parecer
sujeito_verb_adj = [{"LOWER": {"IN": ["homem", "homens", "ele", "eles", "menino", "meninos", "garoto", "garotos", "senhor", "senhores", "moços", "moços", "rapaz", "rapazes", "joão", "cavalheiro", "cavalheiros", "rei", "reis", "esposo", "esposos", "marido", "maridos", "namorado", "namorados"]}}, {'POS': 'VERB', "LEMMA":{"IN":["parecer", "ficar", "tornar"]}}, {'POS': 'ADJ'}]
#Sujeito + Verbo + muito/pouco + Adjetivo / LEMMA tornar, ficar, parecer
sujeito_verb_adv_adj = [{"LOWER": {"IN": ["homem", "homens", "ele", "eles", "menino", "meninos", "garoto", "garotos", "senhor", "senhores", "moços", "moços", "rapaz", "rapazes", "joão", "cavalheiro", "cavalheiros", "rei", "reis", "esposo", "esposos", "marido", "maridos", "namorado", "namorados"]}}, {'POS': 'VERB', "LEMMA":{"IN":["parecer", "ficar", "tornar"]}}, {'POS': "ADV", 'OP': '+'}, {'POS': 'ADJ'}]
#Voce + Verbo + Adjetivo / LEMMA tornar, ficar, parecer
pronoun_verb_adj = [{'POS': 'PRON',"MORPH": {"IN": ['Case=Nom|Number=Sing|Person=3|PronType=Prs']}}, {'POS': 'VERB', "LEMMA":{"IN":["parecer", "ficar", "tornar"]}}, {'POS': 'ADJ', "MORPH": {"IN":["Gender=Masc|Number=Plur", "Gender=Masc|Number=Sing"]}}]
#Voce + Verbo + muito/pouco + Adjetivo / LEMMA tornar, ficar, parecer
pronoun_verb_adv_adj = [{'POS': 'PRON',"MORPH": {"IN": ['Case=Nom|Number=Sing|Person=3|PronType=Prs']}}, {'POS': 'VERB', "LEMMA":{"IN":["parecer", "ficar", "tornar"]}}, {'POS': "ADV", 'OP': '+'}, {'POS': 'ADJ', "MORPH": {"IN":["Gender=Masc|Number=Plur", "Gender=Masc|Number=Sing"]}}]
#Sujeito + Adjetivo
sujeito_adj = [{"LOWER": {"IN": ["homem", "homens", "ele", "eles", "menino", "meninos", "garoto", "garotos", "senhor", "senhores", "moços", "moços", "rapaz", "rapazes", "joão", "cavalheiro", "cavalheiros", "rei", "reis", "esposo", "esposos", "marido", "maridos", "namorado", "namorados"]}}, {"POS":"ADJ"}]
#Sujeito + Pronome +  Verbo (LEMMA) + Adjetivo / Ela se tornou bonito
sujeito_pron_verb_adj = [{"LOWER": {"IN": ["homem", "homens", "ele", "eles", "menino", "meninos", "garoto", "garotos", "senhor", "senhores", "moços", "moços", "rapaz", "rapazes", "joão", "cavalheiro", "cavalheiros", "rei", "reis", "esposo", "esposos", "marido", "maridos", "namorado", "namorados"]}}, {'POS':'PRON'},{'POS': 'VERB', "LEMMA":{"IN":["parecer", "ficar", "tornar"]}}, {'POS': 'ADJ'}]
#Você +Pronome +  Verbo (LEMMA) + Adjetivo / Você se tornou bonito
pronoun_pron_verb_adj = [{'POS': 'PRON',"MORPH": {"IN": ['Case=Nom|Number=Sing|Person=3|PronType=Prs']}}, {'POS':'PRON'},{'POS': 'VERB', "LEMMA":{"IN":["parecer", "ficar", "tornar"]}}, {'POS': 'ADJ', "MORPH": {"IN":["Gender=Masc|Number=Plur", "Gender=Masc|Number=Sing"]}}]
#Sujeito +Pronome +  Verbo (LEMMA) + Adjetivo / Ela se tornou muito bonito
sujeito_pron_verb_adv_adj = [{"LOWER": {"IN": ["homem", "homens", "ele", "eles", "menino", "meninos", "garoto", "garotos", "senhor", "senhores", "moços", "moços", "rapaz", "rapazes", "joão", "cavalheiro", "cavalheiros", "rei", "reis", "esposo", "esposos", "marido", "maridos", "namorado", "namorados"]}}, {'POS':'PRON'}, {'POS': 'VERB', "LEMMA":{"IN":["parecer", "ficar", "tornar"]}}, {'POS': "ADV", 'OP': '+'}, {'POS': 'ADJ'}]
#Você +Pronome +  Verbo (LEMMA) + Adjetivo / Você se tornou muito bonito
pronoun_pron_verb_adv_adj = [{'POS': 'PRON',"MORPH": {"IN": ['Case=Nom|Number=Sing|Person=3|PronType=Prs']}}, {'POS':'PRON'},{'POS': 'VERB', "LEMMA":{"IN":["parecer", "ficar", "tornar"]}}, {'POS': "ADV", 'OP': '+'}, {'POS': 'ADJ', "MORPH": {"IN":["Gender=Masc|Number=Plur", "Gender=Masc|Number=Sing"]}}]
#Sujeito + Auxiliar + Pronome +  Verbo (LEMMA) + Adjetivo
sujeito_aux_pron_verb_adj = [{"LOWER": {"IN": ["homem", "homens", "ele", "eles", "menino", "meninos", "garoto", "garotos", "senhor", "senhores", "moços", "moços", "rapaz", "rapazes", "joão", "cavalheiro", "cavalheiros", "rei", "reis", "esposo", "esposos", "marido", "maridos", "namorado", "namorados"]}}, {'POS': "AUX"}, {'POS':'PRON'}, {'POS': 'VERB', "LEMMA":{"IN":["parecer", "ficar", "tornar"]}}, {'POS': 'ADJ'}]
#Voce + Auxiliar + Pronome +  Verbo (LEMMA) + Adjetivo
pronoun_aux_pron_verb_adj = [{'POS': 'PRON',"MORPH": {"IN": ['Case=Nom|Number=Sing|Person=3|PronType=Prs']}}, {'POS': "AUX"}, {'POS':'PRON'}, {'POS': 'VERB', "LEMMA":{"IN":["parecer", "ficar", "tornar"]}}, {'POS': 'ADJ' , "MORPH": {"IN":["Gender=Masc|Number=Plur", "Gender=Masc|Number=Sing"]}}]
#Voce + Auxiliar + Pronome +  Verbo (LEMMA) + muito/pouco + Adjetivo
pronoun_aux_pron_verb_adv_adj = [{'POS': 'PRON',"MORPH":{"IN":['Case=Nom|Number=Sing|Person=3|PronType=Prs']}},{'POS':'AUX'}, {'POS':'PRON'}, {'POS': 'VERB', "LEMMA":{"IN":["parecer", "ficar", "tornar"]}}, {'POS': "ADV", 'OP': '+'}, {'POS': 'ADJ', "MORPH": {"IN":["Gender=Masc|Number=Plur", "Gender=Masc|Number=Sing"]}}]
#Sujeito + Auxiliar + Pronome +  Verbo (LEMMA) + muito/pouco + Adjetivo
sujeito_aux_pron_verb_adv_adj = [{"LOWER": {"IN": ["homem", "homens", "ele", "eles", "menino", "meninos", "garoto", "garotos", "senhor", "senhores", "moços", "moços", "rapaz", "rapazes", "joão", "cavalheiro", "cavalheiros", "rei", "reis", "esposo", "esposos", "marido", "maridos", "namorado", "namorados"]}}, {'POS': "AUX"}, {'POS':'PRON'}, {'POS': 'VERB', "LEMMA":{"IN":["parecer", "ficar", "tornar"]}}, {'POS': "ADV", 'OP': '+'}, {'POS': 'ADJ'}]
#Sujeito + Verbo + Pronome + Verbo (LEMMA) + Adjetivo
sujeito_verb_pron_verb_adj = [{"LOWER": {"IN": ["homem", "homens", "ele", "eles", "menino", "meninos", "garoto", "garotos", "senhor", "senhores", "moços", "moços", "rapaz", "rapazes", "joão", "cavalheiro", "cavalheiros", "rei", "reis", "esposo", "esposos", "marido", "maridos", "namorado", "namorados"]}}, {'POS': "VERB"}, {'POS':'PRON'},{'POS': 'VERB', "LEMMA":{"IN":["parecer", "ficar", "tornar"]}}, {'POS': 'ADJ'}]
# Você + Verbo + Pronome +  Verbo (LEMMA) + Adjetivo
pronoun_verb_pron_verb_adj = [{'POS': 'PRON',"MORPH": {"IN": ['Case=Nom|Number=Sing|Person=3|PronType=Prs']}}, {'POS':'VERB'}, {'POS':'PRON'}, {'POS': 'VERB', "LEMMA":{"IN":["parecer", "ficar", "tornar"]}}, {'POS': 'ADJ', "MORPH": {"IN":["Gender=Masc|Number=Plur", "Gender=Masc|Number=Sing"]}}]
# Sujeito + Verbo + Pronome +  Verbo (LEMMA) + muito/pouco + Adjetivo
sujeito_verb_pron_verb_adv_adj = [{"LOWER": {"IN": ["homem", "homens", "ele", "eles", "menino", "meninos", "garoto", "garotos", "senhor", "senhores", "moços", "moços", "rapaz", "rapazes", "joão", "cavalheiro", "cavalheiros", "rei", "reis", "esposo", "esposos", "marido", "maridos", "namorado", "namorados"]}}, {'POS': "VERB"}, {'POS':'PRON'},{'POS': 'VERB', "LEMMA":{"IN":["parecer", "ficar", "tornar"]}},{'POS': "ADV", 'OP': '+'}, {'POS': 'ADJ'}]
# Você + Verbo + Pronome +  Verbo (LEMMA) + muito/pouco + Adjetivo
pronoun_verb_pron_verb_adv_adj = [{'POS': 'PRON',"MORPH": {"IN": ['Case=Nom|Number=Sing|Person=3|PronType=Prs']}}, {'POS':'VERB'}, {'POS':'PRON'}, {'POS': 'VERB', "LEMMA":{"IN":["parecer", "ficar", "tornar"]}}, {'POS': "ADV", 'OP': '+'}, {'POS': 'ADJ', "MORPH": {"IN":["Gender=Masc|Number=Plur", "Gender=Masc|Number=Sing"]}}]

# Add the patterns to the matcher under different names
matcher.add('pronoun+aux+adj', patterns=[pronoun_aux_adj])
matcher.add('sujeito_aux_adj', patterns=[sujeito_aux_adj])
matcher.add('pronoun+aux+adv+adj', patterns=[pronoun_aux_adv_adj])
matcher.add('sujeito_aux_adv_adj', patterns=[sujeito_aux_adv_adj])
matcher.add('pronoun_aux_aux_adj ', patterns=[pronoun_aux_aux_adj])
matcher.add('sujeito_aux_aux_adj', patterns=[sujeito_aux_aux_adj])
matcher.add('pronoun_aux_aux_adv_adj', patterns=[pronoun_aux_aux_adv_adj])
matcher.add('sujeito_aux_aux_adv_adj', patterns=[sujeito_aux_aux_adv_adj])
matcher.add('sujeito_aux_verb_adj', patterns=[sujeito_aux_verb_adj])
matcher.add('sujeito_aux_verb_adv_adj', patterns=[sujeito_aux_verb_adv_adj])
matcher.add('pronoun_aux_verb_adj', patterns=[pronoun_aux_verb_adj])
matcher.add('pronoun_aux_verb_adv_adj', patterns=[pronoun_aux_verb_adv_adj])
matcher.add('sujeito_verb_aux_adj', patterns=[sujeito_verb_aux_adj])
matcher.add('sujeito_verb_aux_adv_adj', patterns=[sujeito_verb_aux_adv_adj])
matcher.add('sujeito_adj', patterns=[sujeito_adj])
matcher.add('pronoun_verb_adv_adj', patterns=[pronoun_verb_adv_adj])
matcher.add('pronoun_verb_adj', patterns=[pronoun_verb_adj])
matcher.add('sujeito_verb_adv_adj', patterns=[sujeito_verb_adv_adj])
matcher.add('sujeito_verb_adj', patterns=[sujeito_verb_adj])
matcher.add('sujeito_aux_adv_verb', patterns=[sujeito_aux_adv_verb])
matcher.add('sujeito_aux_verb', patterns=[sujeito_aux_verb])
matcher.add('sujeito_pron_verb_adj', patterns=[sujeito_pron_verb_adj])
matcher.add('pronoun_pron_verb_adj', patterns=[pronoun_pron_verb_adj])
matcher.add('sujeito_pron_verb_adv_adj', patterns=[sujeito_pron_verb_adv_adj])
matcher.add('pronoun_pron_verb_adv_adj', patterns=[pronoun_pron_verb_adv_adj])
matcher.add('pronoun_aux_pron_verb_adj', patterns=[pronoun_aux_pron_verb_adj])
matcher.add('sujeito_aux_pron_verb_adj', patterns=[sujeito_aux_pron_verb_adj])
matcher.add('pronoun_aux_pron_verb_adv_adj', patterns=[pronoun_aux_pron_verb_adv_adj])
matcher.add('sujeito_aux_pron_verb_adv_adj', patterns=[sujeito_aux_pron_verb_adv_adj])
matcher.add('sujeito_verb_pron_verb_adj', patterns=[sujeito_verb_pron_verb_adj])
matcher.add('pronoun_verb_pron_verb_adj', patterns=[pronoun_verb_pron_verb_adj])
matcher.add('pronoun_verb_pron_verb_adv_adj', patterns=[pronoun_verb_pron_verb_adv_adj])
matcher.add('sujeito_verb_pron_verb_adv_adj', patterns=[sujeito_verb_pron_verb_adv_adj])


In [5]:
resultados = []

for index, row in dataframe_sem_gospel.iterrows():
    sentenca = str(row['music_lyric'])
    doc = pln(sentenca)

    results = matcher(doc, as_spans=True)

    for result in results:
      if len(result) > 0:
        #print(pln.vocab[result.label].text, '\t', result)
        resultados.append({'Padrão': pln.vocab[result.label].text, 'Frase': result, 'Letra_Inteira': sentenca})

resultados_df = pd.DataFrame(resultados)
resultados_df.head()
        #displacy.render(doc, style='dep', options={'compact': True, 'distance': 100}, jupyter=True)

,Padrão,Frase,Letra_Inteira
0,pronoun+aux+adj,"(você, tá, louco)","Quer colar comigo, comigo não cola, pra colar ..."
1,pronoun+aux+adj,"(você, tá, louco)","Quer colar comigo, comigo não cola, pra colar ..."
2,pronoun+aux+adj,"(você, é, interesseira)","Japão, Brasil esse mesclado vai te dominar\nPe..."
3,pronoun+aux+adv+adj,"(Você, é, muito, inteligente)","Ei, Miss Simpatia\nMe desculpe mas hoje não va..."
4,sujeito_verb_adj,"(homem, fica, amargo)",Canto pra você\nPorque eu não tenho mais pra q...


In [6]:
from google.colab import files

# Supondo que 'resultados' é uma lista de dicionários ou uma estrutura adequada para criar um DataFrame
resultados_df = pd.DataFrame(resultados)

# Salvar o DataFrame como um arquivo CSV
resultados_df.to_csv('busca_com_POS_masculino.csv', index=False)

# Fazer o download do arquivo CSV
files.download('busca_com_POS_masculino.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>